In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd drive/MyDrive

# 安裝環境

In [ ]:
! git clone https://github.com/WongKinYiu/yolor

In [ ]:
%cd yolor

In [ ]:
! pip install -qr requirements.txt

In [ ]:
!git clone https://github.com/JunnYu/mish-cuda
%cd mish-cuda
!python setup.py build install
%cd ..

# 資料集

In [ ]:
! mv ../plantdoc plantdoc

# 下載 pre-trained model

In [ ]:
! mkdir weights
! pip install gdown
! gdown --id 1Tdn3yqpZ79X7R1Ql0zNlNScB1Dv9Fp76 -O "weights/yolor_p6.pt"

# 修改 data 中的 coco.names 和 coco.yaml

In [ ]:
# 查看類別
with open("plantdoc/train/classes.txt") as f:
  class_names = f.read().splitlines()
    
print(class_names)

In [ ]:
! pwd

In [ ]:
%%writefile data/myself_data.yaml 
# train and val datasets (image directory or *.txt file with image paths)
train: /content/drive/MyDrive/yolor/plantdoc/train/train.txt
val: /content/drive/MyDrive/yolor/plantdoc/train/valid.txt
# test: ../coco/test-dev2017.txt  

# number of classes
nc: 30

# class names
names: ['Apple_Scab_Leaf', 'Apple_leaf', 'Apple_rust_leaf', 'Bell_pepper_leaf_spot', 'Bell_pepper_leaf', 'Blueberry_leaf', 'Cherry_leaf', 'Corn_Gray_leaf_spot', 'Corn_leaf_blight', 'Corn_rust_leaf', 'Peach_leaf', 'Potato_leaf_early_blight', 'Potato_leaf_late_blight', 'Potato_leaf', 'Raspberry_leaf', 'Soyabean_leaf', 'Soybean_leaf', 'Squash_Powdery_mildew_leaf', 'Strawberry_leaf', 'Tomato_Early_blight_leaf', 'Tomato_Septoria_leaf_spot', 'Tomato_leaf_bacterial_spot', 'Tomato_leaf_late_blight', 'Tomato_leaf_mosaic_virus', 'Tomato_leaf_yellow_virus', 'Tomato_leaf', 'Tomato_mold_leaf', 'Tomato_two_spotted_spider_mites_leaf', 'grape_leaf_black_rot', 'grape_leaf']


In [ ]:
! cp data/coco.names data/myself_data.names

In [ ]:
# 修改 myself_data.names 內容
names_file = 'data/myself_data.names'
with open(names_file, 'w') as f:
  for i in class_names:
    f.write('%s\n' % i)

# 修改 cfg 中的 width, height, filters, classes

In [ ]:
myself_yolor_cfg = 'cfg/myself_yolor_p6.cfg'

In [ ]:
! cp cfg/yolor_p6.cfg {myself_yolor_cfg}

In [ ]:
! sed -n -e 1569p -e 1573p -e 1577p -e 1581p -e 1605p -e 1614p -e 1649p -e 1658p -e 1693p -e 1702p -e 1737p -e 1746p {myself_yolor_cfg}

In [ ]:
! sed -i '1569s/255/105/' {myself_yolor_cfg}
! sed -i '1573s/255/105/' {myself_yolor_cfg}
! sed -i '1577s/255/105/' {myself_yolor_cfg}
! sed -i '1581s/255/105/' {myself_yolor_cfg}
! sed -i '1605s/255/105/' {myself_yolor_cfg}
! sed -i '1614s/80/30/' {myself_yolor_cfg}
! sed -i '1649s/255/105/' {myself_yolor_cfg}
! sed -i '1658s/80/30/' {myself_yolor_cfg}
! sed -i '1693s/255/105/' {myself_yolor_cfg}
! sed -i '1702s/80/30/' {myself_yolor_cfg}
! sed -i '1737s/255/105/' {myself_yolor_cfg}
! sed -i '1746s/80/30/' {myself_yolor_cfg}

In [ ]:
! sed -n -e 1569p -e 1573p -e 1577p -e 1581p -e 1605p -e 1614p -e 1649p -e 1658p -e 1693p -e 1702p -e 1737p -e 1746p {myself_yolor_cfg}

# 訓練模型

In [ ]:
!python train.py --batch-size 8 --img 640 640 --data myself_data.yaml --cfg cfg/myself_yolor_p6.cfg --weights 'weights/yolor_p6.pt' --device 0 --name yolor_p6_v1 --hyp hyp.scratch.1280.yaml --epochs 300


In [ ]:
# 可以用多顆 GPU 進行訓練
!python -m torch.distributed.launch --nproc_per_node 2 --master_port 9527 train.py --batch-size 16 --img 640 640 --data data/myself_data.yaml --cfg cfg/myself_yolor_p6.cfg --weights 'weights/yolor_p6.pt' --device 0,1 --sync-bn --name yolor_p6_v1 --hyp hyp.scratch.1280.yaml --epochs 300


# 測試模型

In [ ]:
!python detect.py --source plantdoc/test/images/00_jpg.rf.7fa2b9652948e8c39a51a68ec5c6b70a.jpg --cfg cfg/myself_yolor_p6.cfg --names data/myself_data.names --weights runs/train/yolor_p6_v1/weights/yolor_p6.pt --conf 0.25 --img-size 640 --device 0